In [1]:

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class L2RegularizedLoss(nn.Module):
    def __init__(self, lambda_reg):
        super(L2RegularizedLoss, self).__init__()
        self.lambda_reg = lambda_reg  # Regularization strength

    def forward(self, model_output, target):
        # Calculate the standard loss (e.g., mean squared error)
        standard_loss = torch.nn.functional.mse_loss(model_output, target)

        # Calculate L2 regularization term
        l2_reg = 0.0
        for param in self.parameters():
            l2_reg += torch.norm(param, p=2)  # L2 norm

        # Add L2 regularization to the loss
        total_loss = standard_loss + self.lambda_reg * l2_reg

        return total_loss

# Define the linear ANN model
class LinearANN(nn.Module):
    def __init__(self, input_size, hidden_size_1,hidden_size_2,output_size):
        super(LinearANN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size_1)
        self.activation=nn.ReLU()
        self.layer2=nn.Linear(hidden_size_1,hidden_size_2)
        self.layer3=nn.Linear(hidden_size_2,output_size)

    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)
        x = self.layer2(x)
        x=self.activation(x)
        x=self.layer3(x)
        return x

# Create a custom dataset class for loading data
class CustomDataset(Dataset):
    def __init__(self, X_file_path,Y_file_path,y_layer_num):
        # Load data from the pickle file
        x_f=open(X_file_path, 'rb') 
        y_f=open(Y_file_path,"rb")

        X_data = pickle.load(x_f)
        Y_data=pickle.load(y_f)
        
        
            
        x_torch=torch.Tensor(X_data).to(device)
        y_torch= torch.Tensor(Y_data[:,y_layer_num,:]).to(device)
        
        # Normalize the tensor
        x_torch = torch.nn.functional.normalize(x_torch, p=2, dim=1)
        # Assuming data is a dictionary with keys 'X' and 'y'
        self.inputs = x_torch
        self.targets = y_torch
        
        

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        
        return self.inputs[idx], self.targets[idx]
layers_ann_lst=[]
layers=8
for layer in range(0,layers):
    print("For layers :",layer+1,"\n\n")
    # Set the input size and output size
    input_size = 14382  # Replace X with the actual input size
    hidden_size_1=5000
    hidden_size_2=5000
    output_size = 1000

    # Create an instance of the LinearANN model
    model = LinearANN(input_size,hidden_size_1,hidden_size_2 ,output_size).to(device)

    # Define loss function and optimizer
    criterion = L2RegularizedLoss(lambda_reg=0.001)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Use Adam optimizer

    # Example usage of the data loader for training and testing
    
    x_train_path=f"/kaggle/input/sub-wise-convnext-features-and-mri-dataset/sub1/subject_1_train_X_for_ann.pkl"
    y_train_path=f"/kaggle/input/sub-wise-convnext-features-and-mri-dataset/sub1/subject_1_train_Y_for_ann.pkl"
    x_test_path=f"/kaggle/input/sub-wise-convnext-features-and-mri-dataset/sub1/subject_1_test_X_for_ann.pkl"
    y_test_path=f"/kaggle/input/sub-wise-convnext-features-and-mri-dataset/sub1/subject_1_test_Y_for_ann.pkl"
    train_dataset = CustomDataset(x_train_path,y_train_path,layer)
    test_dataset = CustomDataset(x_test_path,y_test_path,layer)

    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

    # Training loop
    num_epochs = 200
    for epoch in range(num_epochs):
        for inputs, targets in train_loader:

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    layers_ann_lst.append(model)


For layers : 1 


Epoch [1/200], Loss: 0.5715
Epoch [2/200], Loss: 0.6344
Epoch [3/200], Loss: 0.5990
Epoch [4/200], Loss: 0.6499
Epoch [5/200], Loss: 0.5533
Epoch [6/200], Loss: 0.6346
Epoch [7/200], Loss: 0.6349
Epoch [8/200], Loss: 0.5628
Epoch [9/200], Loss: 0.6228
Epoch [10/200], Loss: 0.5605
Epoch [11/200], Loss: 0.5821
Epoch [12/200], Loss: 0.6514
Epoch [13/200], Loss: 0.5777
Epoch [14/200], Loss: 0.5769
Epoch [15/200], Loss: 0.6140
Epoch [16/200], Loss: 0.6387
Epoch [17/200], Loss: 0.5252
Epoch [18/200], Loss: 0.6085
Epoch [19/200], Loss: 0.5806
Epoch [20/200], Loss: 0.6403
Epoch [21/200], Loss: 0.6163
Epoch [22/200], Loss: 0.5767
Epoch [23/200], Loss: 0.6509
Epoch [24/200], Loss: 0.6036
Epoch [25/200], Loss: 0.5515
Epoch [26/200], Loss: 0.6184
Epoch [27/200], Loss: 0.5871
Epoch [28/200], Loss: 0.6259
Epoch [29/200], Loss: 0.6008
Epoch [30/200], Loss: 0.6374
Epoch [31/200], Loss: 0.5431
Epoch [32/200], Loss: 0.5577
Epoch [33/200], Loss: 0.5686
Epoch [34/200], Loss: 0.5997
Epoch

In [2]:
import seaborn as sns
from statistics import mean 
# Testing loop
model.eval()
total_avg_corr=[]
for layer in range(0,8):
    print("For layer",layer+1)
    layer_avg_correff=[]
    with torch.no_grad():
        for inputs, targets in test_loader:

            prediction = layers_ann_lst[layer](inputs)
            print(targets.shape)
            print(prediction.shape)
            for i in range(0,64):
                try:
                    correlation_coefficient = np.corrcoef(targets[i].detach().cpu().numpy(), prediction[i].detach().cpu().numpy().T)[0,1]
                    layer_avg_correff.append(correlation_coefficient)

                except:
                    print("end of data")
                    break
            # Create a scatter plot

    print(mean(layer_avg_correff))
    total_avg_corr.append(mean(layer_avg_correff))
print("Total avg corr:",mean(total_avg_corr))
            # Evaluate your model as needed


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


For layer 1
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([64, 1000])
torch.Size([40, 1000])
torch.Size([40, 1000])
end of data
-0.0009040

In [3]:
# do pca to select data fmri 
# check if sending random is doing something similar 
# fix normalizing over 1 sample
